In [ ]:
!pip install qiskit_optimization qiskit_aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 50.2 MB/s eta 0:00:00


In [ ]:
# Robust library-QAOA runner with fixed SciPyOptimizerWrapper.minimize(**kwargs)
# Single self-contained cell. Paste into Colab/Jupyter and run.

import time, math, itertools, os, sys, inspect, random
import requests
from math import isfinite
import folium
from folium import Popup

# -------------------------
# Problem data (embedded)
# -------------------------
data = {
  "problem_description": {"constraints": {"stops_per_trip": 3}},
  "locations": {
    "hospital": {"name":"Central Hospital","coordinates":{"latitude":29.99512653425452,"longitude":31.68462840171934}},
    "patients":[
      {"id":"DT","coordinates":{"latitude":30.000417586266437,"longitude":31.73960813272627}},
      {"id":"GR","coordinates":{"latitude":30.011344405285193,"longitude":31.747827362371993}},
      {"id":"R2","coordinates":{"latitude":30.030388325206854,"longitude":31.669231198639675}},
      {"id":"R3_2","coordinates":{"latitude":30.030940768851426,"longitude":31.688371339937028}},
      {"id":"IT","coordinates":{"latitude":30.01285635906825,"longitude":31.693811715848444}}
    ]
  }
}

hospital = data["locations"]["hospital"]["coordinates"]
patients = data["locations"]["patients"]
ids = [p["id"] for p in patients]
coord_map = {"H": (hospital["latitude"], hospital["longitude"])}
for p in patients:
    coord_map[p["id"]] = (p["coordinates"]["latitude"], p["coordinates"]["longitude"])

# -------------------------
# OSRM/ORS road-distance matrix (km)
# -------------------------
def lonlat_list(keys):
    return [(coord_map[k][1], coord_map[k][0]) for k in keys]

def fetch_osrm_table(keys, timeout=20):
    coord_str = ";".join(f"{lon:.6f},{lat:.6f}" for lon,lat in lonlat_list(keys))
    url = f"http://router.project-osrm.org/table/v1/driving/{coord_str}?annotations=distance"
    r = requests.get(url, timeout=timeout)
    r.raise_for_status()
    js = r.json()
    if "distances" not in js:
        raise RuntimeError("OSRM response missing 'distances'")
    return [[(v/1000.0 if v is not None else None) for v in row] for row in js["distances"]]

def fetch_ors_table(keys, api_key, timeout=30):
    coords = lonlat_list(keys)
    url = "https://api.openrouteservice.org/v2/matrix/driving-car"
    headers = {"Authorization": api_key, "Content-Type": "application/json"}
    body = {"locations": coords, "metrics": ["distance"], "units": "m"}
    r = requests.post(url, headers=headers, json=body, timeout=timeout)
    r.raise_for_status()
    js = r.json()
    if "distances" not in js:
        raise RuntimeError("ORS response missing 'distances'")
    return [[(v/1000.0 if v is not None else None) for v in row] for row in js["distances"]]

def build_dist_matrix(ors_api_key=None):
    points = ["H"] + ids
    mat = None
    try:
        print("Requesting OSRM table...")
        mat = fetch_osrm_table(points)
        print("OSRM table received.")
    except Exception as e:
        print("OSRM failed:", e)
        mat = None
    if mat is not None and any(mat[i][j] is None for i in range(len(mat)) for j in range(len(mat))):
        if ors_api_key:
            print("OSRM incomplete — trying ORS fallback...")
            mat2 = fetch_ors_table(points, ors_api_key)
            if any(mat2[i][j] is None for i in range(len(mat2)) for j in range(len(mat2))):
                raise RuntimeError("ORS also has unreachable pairs.")
            mat = mat2
        else:
            raise RuntimeError("OSRM matrix contains unreachable pairs and no ORS_API_KEY provided.")
    if mat is None:
        raise RuntimeError("No routing matrix available.")
    dist = {}
    for i,a in enumerate(points):
        dist[a] = {}
        for j,b in enumerate(points):
            v = mat[i][j]
            if v is None:
                raise RuntimeError(f"Unroutable pair {a} -> {b}")
            dist[a][b] = float(v)
    return dist

ORS_API_KEY = os.environ.get("ORS_API_KEY")
dist = build_dist_matrix(ors_api_key=ORS_API_KEY)

# -------------------------
# Candidate subsets and subset_costs (best intra-trip permutation)
# -------------------------
max_stops = data["problem_description"]["constraints"]["stops_per_trip"]
all_subsets = []
subset_cost = {}
for r in range(1, max_stops+1):
    for comb in itertools.combinations(ids, r):
        all_subsets.append(tuple(comb))
        best = math.inf
        for perm in itertools.permutations(comb):
            c = dist["H"][perm[0]]
            for k in range(len(perm)-1):
                c += dist[perm[k]][perm[k+1]]
            c += dist[perm[-1]]["H"]
            if c < best:
                best = c
        subset_cost[tuple(comb)] = best

# prune subsets while preserving coverage to keep qubit count small
MAX_VARS = 25
sorted_subs = sorted(all_subsets, key=lambda s: subset_cost[s])
selected_subs = []
covered = set()
for s in sorted_subs:
    if any(p not in covered for p in s):
        selected_subs.append(s); covered.update(s)
    if len(selected_subs) >= MAX_VARS:
        break
idx = 0
while set(ids) - covered and idx < len(sorted_subs):
    s = sorted_subs[idx]
    if s not in selected_subs:
        selected_subs.append(s); covered.update(s)
    idx += 1
if set(ids) - covered:
    raise RuntimeError("Pruning failed to preserve coverage — increase MAX_VARS")

subsets = selected_subs

print(f"Using {len(subsets)} binary variables (subset candidates). Subsets:")
for i,s in enumerate(subsets,1):
    print(f" {i}. {s} cost={subset_cost[s]:.6f} km")

# -------------------------
# Build QuadraticProgram later after import check
# -------------------------
# -------------------------
# Try imports (qiskit_optimization, QAOA, Sampler/Aer)
# -------------------------
try:
    from qiskit_optimization import QuadraticProgram
    from qiskit_optimization.algorithms import MinimumEigenOptimizer
    from qiskit_optimization.minimum_eigensolvers import QAOA
    try:
        from qiskit_aer.primitives import SamplerV2
        from qiskit_aer import AerSimulator
    except Exception:
        try:
            from qiskit.primitives import Sampler as SamplerV2
            from qiskit_aer import AerSimulator
        except Exception:
            from qiskit_aer import AerSimulator
            SamplerV2 = None
except Exception as e:
    import traceback
    traceback.print_exc()
    raise RuntimeError("Required Qiskit modules not importable. Consider clean reinstall.") from e

# Build QuadraticProgram
qp = QuadraticProgram()
for s in subsets:
    qp.binary_var(name="y_" + "_".join(s))
qp.minimize(linear={ "y_" + "_".join(s): subset_cost[s] for s in subsets })
for p in ids:
    vars_for_p = ["y_" + "_".join(s) for s in subsets if p in s]
    qp.linear_constraint(linear={v: 1 for v in vars_for_p}, sense='==', rhs=1, name=f"cover_{p}")

# -------------------------
# Instantiate sampler robustly
# -------------------------
sampler = None
sampler_errs = []
if 'SamplerV2' in globals() and SamplerV2 is not None:
    constructors = [
        lambda: SamplerV2(mode=AerSimulator()),
        lambda: SamplerV2(backend=AerSimulator()),
        lambda: SamplerV2(),
    ]
else:
    constructors = [lambda: AerSimulator()]

for fn in constructors:
    try:
        sampler = fn()
        print("Sampler instantiated using constructor:", fn)
        break
    except Exception as e:
        sampler_errs.append(e)
if sampler is None:
    print("Sampler constructors tried and failed. Errors:")
    for i,e in enumerate(sampler_errs,1):
        print(f"[{i}] {type(e).__name__}: {e}")
    raise RuntimeError("Unable to create sampler instance.")

# -------------------------
# Build optimizer or wrapper (fix: minimize accepts **kwargs)
# -------------------------
optimizer = None
try:
    from qiskit.algorithms.optimizers import SPSA, COBYLA
    optimizer = SPSA(maxiter=200)
    print("Using qiskit SPSA optimizer.")
except Exception:
    try:
        from qiskit.algorithms.optimizers import COBYLA
        optimizer = COBYLA(maxiter=200)
        print("Using qiskit COBYLA optimizer.")
    except Exception:
        optimizer = None

if optimizer is None:
    # SciPy wrapper with robust minimize signature
    from scipy.optimize import minimize as scipy_minimize
    class SciPyOptimizerWrapper:
        def __init__(self, method='COBYLA', options=None):
            self.method = method
            self.options = options or {"maxiter":200}
        # used by some QAOA implementations
        def optimize(self, num_vars, objective, initial_point=None):
            def obj_for_scipy(x):
                val = objective(x)
                if isinstance(val, (tuple, list)):
                    return float(val[0])
                return float(val)
            x0 = initial_point if initial_point is not None else [0.0]*num_vars
            res = scipy_minimize(obj_for_scipy, x0, method=self.method, options=self.options)
            # return (opt_params, opt_value, nfev)
            return res.x, float(res.fun), getattr(res, "nfev", None)
        # KEY FIX: accept arbitrary kwargs (e.g., bounds) forwarded to scipy_minimize
        def minimize(self, fun, x0, **kwargs):
            # kwargs may include bounds, method, options, jac, etc.
            method = kwargs.pop("method", self.method)
            options = kwargs.pop("options", self.options)
            res = scipy_minimize(fun, x0, method=method, options=options, **kwargs)
            return res
    optimizer = SciPyOptimizerWrapper(method='COBYLA', options={"maxiter":200})
    print("Using SciPy-based optimizer wrapper (minimize accepts kwargs).")

# -------------------------
# Inspect QAOA signature & attempt instantiation with optimizer and sampler
# -------------------------
import inspect
qaoa_sig = inspect.signature(QAOA.__init__)
params = [p for p in qaoa_sig.parameters.keys() if p != 'self']
print("QAOA.__init__ params:", params)

qaoa = None
qaoa_errors = []
attempts = [
    {"sampler": sampler, "optimizer": optimizer, "reps": 2},
    {"optimizer": optimizer, "sampler": sampler, "reps": 2},
    {"optimizer": optimizer, "reps": 2, "sampler": sampler},
    {"sampler": sampler, "optimizer": optimizer},
    {"optimizer": optimizer},
]

for kwargs in attempts:
    try:
        qaoa = QAOA(**kwargs)
        print("QAOA instantiated with kwargs:", kwargs)
        break
    except Exception as e:
        qaoa_errors.append((kwargs, e))

if qaoa is None:
    print("QAOA instantiation attempts failed. Summary:")
    for kw,e in qaoa_errors:
        print("Attempt:", kw)
        print(" ->", type(e).__name__, e)
    raise RuntimeError("Could not instantiate QAOA with tested signatures. Paste the above errors and I'll adapt.")

# -------------------------
# MinimumEigenOptimizer wrapper & solve
# -------------------------
try:
    meo = MinimumEigenOptimizer(min_eigen_solver=qaoa)
except TypeError:
    meo = MinimumEigenOptimizer(qaoa)

print("Running MinimumEigenOptimizer (QAOA) ...")
t0 = time.time()
result = meo.solve(qp)
t1 = time.time()
elapsed = t1 - t0

print(f"Finished in {elapsed:.3f} s. Status: {result.status}")
print(f"QuadraticProgram objective (raw): {result.fval:.6f}")

chosen = [s for s in subsets if abs(result.variables_dict.get("y_" + "_".join(s), 0) - 1) < 1e-6]
print("Chosen subsets (y_s=1):")
for s in chosen:
    print("  ", s, f"cost={subset_cost[s]:.6f} km")
total_km = sum(subset_cost[s] for s in chosen)
print(f"Reconstructed total distance (sum of chosen subset costs) = {total_km:.6f} km")

# -------------------------
# Visualize chosen trips on Folium map (OSRM geometry for display only)
# -------------------------
m = folium.Map(location=[coord_map["H"][0], coord_map["H"][1]], zoom_start=12)
folium.Marker([coord_map["H"][0], coord_map["H"][1]], popup=Popup("Hospital"), icon=folium.Icon(color="red")).add_to(m)
for pid in ids:
    folium.Marker([coord_map[pid][0], coord_map[pid][1]], popup=Popup(pid), icon=folium.Icon(color="blue")).add_to(m)

def osrm_route_coords(latlon_points):
    coord_pairs = ["{:.6f},{:.6f}".format(lon, lat) for lat, lon in latlon_points]
    coords_str = ";".join(coord_pairs)
    url = f"http://router.project-osrm.org/route/v1/driving/{coords_str}?overview=full&geometries=geojson"
    try:
        r = requests.get(url, timeout=10); r.raise_for_status()
        js = r.json()
        if "routes" in js and len(js["routes"])>0:
            coords = js["routes"][0]["geometry"]["coordinates"]
            return [(lat,lon) for lon,lat in coords]
    except Exception:
        return None
    return None

colors = ["blue","green","purple","orange","darkred","cadetblue","darkgreen","pink"]
for i,s in enumerate(chosen):
    best_perm=None; bestc=float('inf')
    for perm in itertools.permutations(s):
        c = dist["H"][perm[0]]
        for k in range(len(perm)-1):
            c += dist[perm[k]][perm[k+1]]
        c += dist[perm[-1]]["H"]
        if c < bestc:
            bestc = c; best_perm = perm
    visit = [coord_map["H"]] + [coord_map[p] for p in best_perm] + [coord_map["H"]]
    geom = osrm_route_coords(visit)
    if geom:
        folium.PolyLine(locations=geom, color=colors[i%len(colors)], weight=5, tooltip=f"Trip {i+1}: {list(best_perm)} ({bestc:.2f} km)").add_to(m)
    else:
        folium.PolyLine(locations=visit, color=colors[i%len(colors)], weight=3, tooltip=f"Trip {i+1}: {list(best_perm)} ({bestc:.2f} km)").add_to(m)

out_html = "qaoa_library_signature_fixed_map.html"
m.save(out_html)
print("Map saved to", out_html)
try:
    display(m)
except Exception:
    pass

Requesting OSRM table...
OSRM table received.
Using 5 binary variables (subset candidates). Subsets:
 1. ('R3_2',) cost=18.195100 km
 2. ('IT',) cost=18.941600 km
 3. ('R2',) cost=21.309800 km
 4. ('DT',) cost=22.822100 km
 5. ('DT', 'GR', 'R3_2') cost=28.456300 km
Sampler instantiated using constructor: <function <lambda> at 0x7cce5af1e7a0>
Using SciPy-based optimizer wrapper (minimize accepts kwargs).
QAOA.__init__ params: ['sampler', 'optimizer', 'reps', 'initial_state', 'mixer', 'initial_point', 'aggregation', 'callback', 'pass_manager']
QAOA instantiated with kwargs: {'sampler': <qiskit_aer.primitives.sampler_v2.SamplerV2 object at 0x7cce5ae47c80>, 'optimizer': <__main__.SciPyOptimizerWrapper object at 0x7cce5ae47bf0>, 'reps': 2}
Running MinimumEigenOptimizer (QAOA) ...


/usr/local/lib/python3.12/dist-packages/qiskit_optimization/minimum_eigensolvers/qaoa.py:122: UserWarning: Using Sampler V2 (other than StatevectorSampler) without a pass_manager may result in an error. Consider providing a pass_manager for proper circuit transpilation.
  super().__init__(


Finished in 0.607 s. Status: OptimizationResultStatus.SUCCESS
QuadraticProgram objective (raw): 68.707700
Chosen subsets (y_s=1):
   ('IT',) cost=18.941600 km
   ('R2',) cost=21.309800 km
   ('DT', 'GR', 'R3_2') cost=28.456300 km
Reconstructed total distance (sum of chosen subset costs) = 68.707700 km
Map saved to qaoa_library_signature_fixed_map.html
